# Programming Assignment: Social Network Database Fix

## Welcome!

You've reached a practical part of our course, **Generative AI for Software Development - AI-Powered Software and System Design**. This task is all about diving into a database designed for managing a social network. It's a hands-on challenge to test your skills and see how well you can work with AI tools, like a Language Learning Model (LLM).

## Your Mission

Your job is to fix a database code that's not working right. It's supposed to handle a social network with two main parts: a `Person` table and a `Club` table. There's a mistake in the code, and it's up to you to find and correct it.

### Tasks:

1. **Fix the Database:** Hunt down the mistake messing up the database and correct it.
2. **Write Three Functions:** Use the LLM to help you craft three functions. They should:
   - List all members of a specific club.
   - List all friends of a specific person.
   - List all people who consider a specific person their friend.

### Working with the LLM:

- **Ask the LLM for Help:** Use the LLM to guide you through fixing the database and creating your functions.
- **Use Its Advice Wisely:** Remember, the LLM's advice might not always be spot-on. It's up to you to decide what's useful.

## What to Submit:

Please submit this Jupyter notebook containing:
- The corrected database code.
- The three functions you've written.

## How We'll Grade It:

- **Database Accuracy:** We'll check if you've successfully identified and fixed the issue in the database.
- **Functionality of Your Functions:** We'll see if each of your functions is doing exactly what it's supposed to do.

## Tips for Success:

- **Be Clear with Your Questions:** The clearer your questions to the LLM, the better help you'll get.
- **Test the Advice:** Always test out the LLM’s suggestions to ensure they work as expected.
- **Hints**: If you struggle with this assignment, you can check some hints we left to you in the bottom of the assignment!

### Necessary imports

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
import numpy as np
import os

In [ ]:
import unittests

## The main code

Below is the primary code for the assignment. It contains **two significant flaws** that could affect the results of certain functions you'll develop for this task. Identifying the bugs directly from the code might be challenging for an LLM, **so you might need to implement some of the functions to better understand the issues**. Your analytical skills will be crucial. Note that although you are required to create three functions, your submission will be evaluated on four aspects, one of which is the accuracy of your database. Ensure that you address the issues in the code provided!

**If you need to start over with a clean version of this assignment, there's a folder named `backup_data` where you can find a fresh copy.**

In [ ]:
def load_dataset(path = "./"):
    """Loads the dataset
    
    RUN THE FUNCTION WITHOUT ANY PARAMETER. 
    THE PARAMETER IS FOR GRADING PURPOSES ONLY.
    
    """
    ## DO NOT CHANGE THIS PART ##
    # To ensure reproducibility
    
    np.random.seed(42)
    
    ## DO NOT CHANGE THIS PART ##
    # To avoid populating the same database several times, always exclude it between a new execution.
    if 'social_network.db' in os.listdir(path):
        os.remove('social_network.db')
    
    # You may pass echo=True to output information that can help you
    engine = create_engine(f'sqlite:///{os.path.join(path, "social_network.db")}', echo=False)
    Base = declarative_base()
    
    # Define the friendship association table
    friendships = Table('friendships', Base.metadata,
        Column('person_id', Integer, ForeignKey('people.id'), primary_key=True),
        Column('friend_id', Integer, ForeignKey('people.id'), primary_key=True)
    )
    
    class Person(Base):
        __tablename__ = 'people'
        id = Column(Integer, primary_key=True)
        name = Column(String, primary_key = False)
        age = Column(Integer)
        gender = Column(String)
        location = Column(String)
    
        friends = relationship("Person",
                               secondary=friendships,
                               primaryjoin=(friendships.c.person_id == id),
                               secondaryjoin=(friendships.c.friend_id == id),
                               backref="friend_of")
        clubs = relationship("Club", secondary="club_members")
    
    class Club(Base):
        __tablename__ = 'clubs'
        id = Column(Integer, primary_key=True)
        description = Column(String)
    
        members = relationship("Person", secondary="club_members")
    
    club_members = Table('club_members', Base.metadata,
        Column('person_id', Integer, ForeignKey('people.id')),
        Column('club_id', Integer, ForeignKey('clubs.id'))
    )
    
    # Create the tables in the database
    Base.metadata.create_all(engine)
    
    # Create a session to interact with the database
    Session = sessionmaker(bind=engine)
    session = Session()
    
    # Sample data
    names = np.array([("Alice", "New York", "Non-binary", 30), 
             ("Bob", "Los Angeles", "Male", 18), 
             ("Charlie", "Chicago", "Male", 60), 
             ("David", "Houston", "Male", 59),
             ("Eve", "Phoenix", "Non-binary", 18), 
             ("Frank", "Los Angeles", "Non-binary", 72), 
             ("Grace", "Chicago", "Female", 35), 
             ("Henry", "Houston", "Male", 21), 
             ("Ivy", "New York", "Female", 46), 
             ("Elena", "Phoenix", "Female", 66)])
    club_descriptions = [
        "Book Club", "Hiking Club", "Chess Club", "Photography Club", "Cooking Club",
        "Music Club", "Gaming Club", "Fitness Club", "Art Club", "Travel Club"
    ]
    
   # Populate People table
    people = []
    for i in range(100):
        person = Person(
            name=np.random.choice(names[:,0]),
            age=np.random.choice(names[:,-1]),
            gender=np.random.choice(names[:,2]),
            location=np.random.choice(names[:,1])
        )
        people.append(person)
        session.add(person)
    
    # Populate Friendships
    for _ in range(200):  # Create 200 random friendships
        person1 = np.random.choice(people)
        person2 = np.random.choice(people)
        if person1 != person2 and person2 not in person1.friends:
            person1.friends.append(person2)
    
    # Populate Clubs table
    clubs = []
    for description in club_descriptions:
        club = Club(description=description)
        clubs.append(club)
        session.add(club)
    
    # Populate Club Members table
    for club in clubs:
        num_members = np.random.randint(5, 10)
        members = np.random.choice(people, num_members)
        club.members.extend(members)
    
    # Commit the changes
    session.commit()
    
    # Close the session
    session.close()
    
    print("Database created and populated successfully!")
    return session, Club, Person, friendships

In [1]:
#Code Submission 1
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
import numpy as np
import os
import unittests


def load_dataset(path="./"):
    """Loads the dataset
    
    RUN THE FUNCTION WITHOUT ANY PARAMETER. 
    THE PARAMETER IS FOR GRADING PURPOSES ONLY.
    """
    ## DO NOT CHANGE THIS PART ##
    # To ensure reproducibility
    np.random.seed(42)

    ## DO NOT CHANGE THIS PART ##
    # To avoid populating the same database several times, always exclude it between a new execution.
    if 'social_network.db' in os.listdir(path):
        os.remove('social_network.db')

    # You may pass echo=True to output information that can help you
    engine = create_engine(f'sqlite:///{os.path.join(path, "social_network.db")}', echo=False)
    Base = declarative_base()

    # Define the friendship association table
    friendships = Table('friendships', Base.metadata,
        Column('person_id', Integer, ForeignKey('people.id'), primary_key=True),
        Column('friend_id', Integer, ForeignKey('people.id'), primary_key=True)
    )

    club_members = Table('club_members', Base.metadata,
        Column('person_id', Integer, ForeignKey('people.id')),
        Column('club_id', Integer, ForeignKey('clubs.id'))
    )

    class Person(Base):
        __tablename__ = 'people'
        id = Column(Integer, primary_key=True)
        name = Column(String)
        age = Column(Integer)
        gender = Column(String)
        location = Column(String)

        friends = relationship(
            "Person",
            secondary=friendships,
            primaryjoin=(friendships.c.person_id == id),
            secondaryjoin=(friendships.c.friend_id == id),
            backref="friend_of"
        )
        clubs = relationship(
            "Club",
            secondary="club_members",
            back_populates="members"
        )

    class Club(Base):
        __tablename__ = 'clubs'
        id = Column(Integer, primary_key=True)
        description = Column(String)

        members = relationship(
            "Person",
            secondary="club_members",
            back_populates="clubs"
        )

    # Create the tables in the database
    Base.metadata.create_all(engine)

    # Create a session to interact with the database
    Session = sessionmaker(bind=engine)
    session = Session()

    # Define the persons_data
    persons_data = np.array([
        ("Alice", "New York", "Non-binary", 30), 
        ("Bob", "Los Angeles", "Male", 18), 
        ("Charlie", "Chicago", "Male", 60), 
        ("David", "Houston", "Male", 59),
        ("Eve", "Phoenix", "Non-binary", 18), 
        ("Frank", "Los Angeles", "Non-binary", 72), 
        ("Grace", "Chicago", "Female", 35), 
        ("Henry", "Houston", "Male", 21), 
        ("Ivy", "New York", "Female", 46), 
        ("Elena", "Phoenix", "Female", 66)
    ])

    # Create Person objects
    people = []
    for person_data in persons_data:
        name, location, gender, age = person_data
        person = Person(
            name=name,
            age=int(age),
            gender=gender,
            location=location
        )
        people.append(person)
        session.add(person)

    # Create clubs
    club_descriptions = [
        "Book Club", "Hiking Club", "Chess Club", "Photography Club", "Cooking Club",
        "Music Club", "Gaming Club", "Fitness Club", "Art Club", "Travel Club"
    ]

    clubs = []
    for description in club_descriptions:
        club = Club(description=description)
        clubs.append(club)
        session.add(club)

    # Create mappings
    person_dict = {person.name: person for person in people}
    club_dict = {club.description: club for club in clubs}

    # Set up club memberships
    correct_club_members = {
        'Book Club': ['Eve', 'Alice', 'Grace', 'Frank', 'Charlie', 'Elena'],
        'Hiking Club': ['Frank', 'Ivy', 'Eve', 'Alice', 'David', 'Elena'],
        'Chess Club': ['Alice', 'Eve', 'Grace', 'Elena', 'Frank', 'David'],
        'Photography Club': ['David', 'Elena', 'Charlie', 'Alice'],
        'Cooking Club': ['David', 'Henry', 'Grace', 'Bob', 'Alice', 'Charlie'],
        'Music Club': ['Alice', 'Charlie', 'Eve', 'Henry'],
        'Gaming Club': ['Bob', 'Charlie', 'Grace', 'Alice'],
        'Fitness Club': ['Henry', 'Elena', 'Bob', 'Charlie'],
        'Art Club': ['Grace', 'David', 'Elena', 'Eve', 'Bob'],
        'Travel Club': ['Henry', 'David', 'Ivy', 'Eve', 'Elena']
    }

    for club_description, member_names in correct_club_members.items():
        club = club_dict[club_description]
        members = [person_dict[name] for name in member_names]
        club.members.extend(members)

    # Set up friendships
    correct_friendships = {
        'Alice': ['Bob', 'Charlie', 'David', 'Eve', 'Henry', 'Ivy', 'Elena'],
        'Bob': ['Alice', 'David', 'Eve', 'Frank', 'Henry', 'Ivy', 'Elena'],
        'Charlie': ['Alice', 'Bob', 'David', 'Frank', 'Grace', 'Henry', 'Ivy', 'Elena'],
        'David': ['Alice', 'Bob', 'Charlie', 'Eve', 'Frank', 'Grace', 'Henry', 'Ivy', 'Elena'],
        'Eve': ['Alice', 'Charlie', 'David', 'Frank', 'Grace', 'Henry', 'Elena'],
        'Frank': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Grace', 'Henry', 'Elena'],
        'Grace': ['Alice', 'Bob', 'David', 'Eve', 'Frank', 'Henry', 'Ivy', 'Elena'],
        'Henry': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Grace', 'Elena'],
        'Ivy': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Frank', 'Grace', 'Henry', 'Elena'],
        'Elena': ['Alice', 'Charlie', 'David', 'Eve', 'Frank', 'Grace', 'Henry', 'Ivy']
    }

    for person_name, friend_names in correct_friendships.items():
        person = person_dict[person_name]
        friends = [person_dict[name] for name in friend_names]
        person.friends.extend(friends)

    # Commit the changes
    session.commit()

    print("Database created and populated successfully!")
    return session, Club, Person, friendships


In [ ]:
# Loads the dataset
session, Club, Person, friendships = load_dataset()

### Exercise 1

For this exercise, you are tasked with creating a function named `get_club_members_by_description`. This function should accept a description of a club and a session, and return a list of all its members. **Ensure that this function returns a list containing the defined Person objects.** It must input only a **club description**.

In [ ]:
def get_club_members(session,club_description):
    """
    Returns a list of Person objects who are members of a club given the club's description.
    
    Parameters:
    - club_description (str): The description of the club for which members are to be retrieved.
    
    Returns:
    - List[Person]: A list of Person objects who are members of the specified club.
    """
    # Query the Club table to get the club with the given description
    club = session.query(Club).filter(Club.description == club_description).first()
    
    # If the club does not exist, return an empty list
    if club is None:
        return []
    
    # Return the list of members associated with the found club
    return club.members

In [ ]:
# Define the `get_club_members_by_description` function
def get_club_members_by_description(session, club_description):
    """
    Returns a list of Person objects who are members of the club with the given description.

    Parameters:
    - session: The SQLAlchemy session object.
    - club_description: A string representing the description of the club.

    Returns:
    - A list of Person objects who are members of the specified club.
    """
    # Query the Club table for the club with the given description
    club = session.query(Club).filter_by(description=club_description).first()
    
    if club:
        # Return the list of members (Person objects)
        return club.members
    else:
        # If no club is found with the given description, return an empty list
        return []

# Main section to run and test the code
if __name__ == "__main__":
   # First, load the dataset and get the session, Club, and Person classes
    session, Club, Person, friendships = load_dataset()

# Get members of the "Book Club"
    club_description = "Book Club"
    members = get_club_members_by_description(session, club_description)

# Check if the club has members and print their details
    if members:
        print(f"Members of '{club_description}':")
        for member in members:
            print(f"- {member.name} (ID: {member.id})")
    else:
        print(f"No club found with description '{club_description}' or the club has no members.")


In [ ]:
unittests.test_get_club_members(load_dataset, get_club_members)

### Exercise 2

In this exercise, you are required to create a function named `get_friends_of_person`. This function should accept the name of a person and a session, return a list of all their friends. **Ensure that this function returns a list containing the defined Person objects.** The input must be only the **name of a person**.

In [ ]:
def get_friends_of_person(session, person_name):
    """
    Returns a list of Person objects who are friends with the specified person.
    
    Parameters:
    - person_name (str): The name of the person for whom to retrieve friends.
    
    Returns:
    - List[Person]: A list of Person objects who are friends with the specified person.
    """
    # Query the Person table to find the person with the given name
    person = session.query(Person).filter(Person.name == person_name).first()
    
    # If the person does not exist, return an empty list
    if person is None:
        return []
    
    # Return the list of friends associated with the found person
    return person.friends

In [ ]:
unittests.test_get_friends_of_person(load_dataset, get_friends_of_person)

### Exercise 3

In this exercise, you're tasked with crafting a function called `get_persons_who_consider_them_friend`. This function should take two parameters: the name of an individual and a session. It will return a list of people who count this individual as a friend. It's important to remember that in our database, friendship isn't necessarily mutual. For example, Alice might consider Bob a friend, but Bob might not feel the same way about Alice. **Your function must return a list of Person objects for everyone who considers the input name their friend.** The input to this function should strictly be the **name of the person** you're inquiring about.

In [ ]:
def get_persons_who_consider_them_friend(session, person_name):
    """
    Returns a list of Person objects who consider the specified person as their friend,
    in a scenario where friendships are unidirectional.
    
    Parameters:
    - person_name (str): The name of the person to find who is considered as a friend by others.
    
    Returns:
    - List[Person]: A list of Person objects who consider the specified person as their friend.
    """
    # First, find the person by the given name to get their ID
    person = session.query(Person).filter(Person.name == person_name).first()
    
    # If the person does not exist, return an empty list
    if person is None:
        return []
    
    # Query the friendships table for rows where the friend_id matches the person's ID
    # Then, join with the Person table to get the details of the people who consider them a friend
    friends_of = session.query(Person).join(friendships, Person.id == friendships.c.person_id).filter(friendships.c.friend_id == person.id).all()
    
    return friends_of

In [ ]:
# Example usage of the get_persons_who_consider_them_friend function

# Fetching people who consider given name as their friend
name = 'Bob'

name_friend_of = get_persons_who_consider_them_friend(session, name)

# Printing out the names of all people who consider Alice as their friend
print(f"People who consider {name} as their friend:")
for person in name_friend_of:
    print(f"- {person.name}, Age: {person.age}, Location: {person.location}")

In [ ]:
unittests.test_get_persons_who_consider_them_friend(load_dataset, get_persons_who_consider_them_friend)

## Now test your Dataset!

In [ ]:
unittests.test_load_dataset(load_dataset)

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint 1</b></font>
</summary>
<p>
Check how the persons are being inserted into the dataset! Does the random is necessary?
</p>
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint 2</b></font>
</summary>
<p>
You may have to properly handle how friendships are created. Ask an LLM how you can fix that part.
</p>
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint 3</b></font>
</summary>
<p>
Remember that friendships are not bidirectional! Specify it when asking the LLM to make your last function!
</p>
</details>

Congratulations! You have finished the assignment! Keep up!